# Stock Market

In this project, I worked with the data from the S&P500 Index. The S&P500 is a stock market index. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv. 

The columns of the dataset are:
* **Date** -- The date of the record.
* **Open** -- The opening price of the day (when trading starts).
* **High** -- The highest trade price during the day.
* **Low** -- The lowest trade price during the day.
* **Close** -- The closing price for the day (when trading is finished).
* **Volume** -- The number of shares traded.
* **Adj Close** -- The daily closing price, adjusted retroactively to include any corporate actions. Read more here.

In [23]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv('sphist.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [3]:
# Convert Date colum into datetime format
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null datetime64[ns]
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: datetime64[ns](1), float64(6)
memory usage: 907.4 KB


In [4]:
sorted_df = df.sort_values(by=['Date'])
sorted_df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


## Generating Indicators


Here are some indicators that are interesting to generate for each row:
* The average price from the past 5 days.
* The average price for the past 30 days.
* The average price for the past 365 days.
* The ratio between the average price for the past 5 days, and the average price for the past 365 days.
* The standard deviation of the price over the past 5 days.
* The standard deviation of the price over the past 365 days.
* The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

In [5]:
sorted_df = sorted_df.shift(periods=1, freq=None)
sorted_df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,NaT,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16587,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16586,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16585,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98


In [6]:
sorted_df['day_5'] = sorted_df['Close'].rolling(5, win_type='triang').mean()
sorted_df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,day_5
16589,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN
16587,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN
16586,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN
16585,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN
16584,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,16.910000
16583,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.982222
16582,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,17.031111
16581,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.018889
16580,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.955556


## Splitting up the data

Since you're computing indicators that use historical data, there are some rows where there isn't enough historical data to generate them. Some of the indicators use 365 days of historical data, and the dataset starts on 1950-01-03. Thus, any rows that fall before 1951-01-03 don't have enough historical data to compute all the indicators. You'll need to remove these rows before you split the data.

In [7]:
df = sorted_df[sorted_df['Date'] > datetime(year=1951, month=1, day=2)]
df = df.dropna(axis=0)

In [11]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
day_5        0
dtype: int64

In [14]:
train = df[df['Date'] < datetime(year=2013, month=1, day=1)]
test = df[df['Date'] >= datetime(year=2013, month=1, day=1)]

train.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5
16338,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,20.508889
16337,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.644445
16336,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,20.738890
16335,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,20.833334
16334,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,20.906667


## Making predictions

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'day_5']
target = ['Close']

lr = LinearRegression()
lr.fit(train[features], train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [18]:
lr.coef_

array([[-2.00847498e-14,  2.56360005e-14, -4.01079250e-15,
         5.00000000e-01, -7.44746800e-20,  5.00000000e-01,
         1.38282358e-16]])

In [19]:
lr.intercept_

array([5.0079052e-11])

In [21]:
test_predictions = lr.predict(test[features])

mse = mean_squared_error(test[target], test_predictions)
mse

4.375209096989726e-20

In [24]:
rmse = np.sqrt(mse)
rmse

2.0917000494788267e-10

I can improve the error of this model significantly, though. Some ideas that might be helpful:

* The average volume over the past five days.
* The average volume over the past year.
* The ratio between the average volume for the past five days, and the average volume for the past year.
* The standard deviation of the average volume over the past five days.
* The standard deviation of the average volume over the past year.
* The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
* The year component of the date.
* The ratio between the lowest price in the past year and the current price.
* The ratio between the highest price in the past year and the current price.
* The month component of the date.
* The day of week.
* The day component of the date.
* The number of holidays in the prior month.